In [1]:
import numpy as np
import tensorflow as tf
import threading
import cv2

import PIL
import matplotlib.pyplot as plt

In [2]:
SCAN_RESOLUTION: int = (384, 288)

In [3]:
def capture_video_thread_function():
    capture = cv2.VideoCapture(0)
    while True:
        ret, frame = capture.read()
        gray = cv2.resize(frame, SCAN_RESOLUTION)
        gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
        cv2.imshow('Grayscale', gray)

        cv2.imshow('Face Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.imwrite("example.jpg", frame)
            break
    print(frame.shape)
    capture.release()
    cv2.destroyAllWindows()

In [4]:
thread = threading.Thread(target = capture_video_thread_function)
thread.start()

In [5]:
def load_img(img_path:str, res:tuple[int, int] = None) -> tf.Tensor:
    img = PIL.Image.open(img_path)
    if res:
        img = img.resize(res)
    return np.array(img)

In [6]:
def to_grayscale(img: tf.Tensor) -> np.array:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray.astype("uint32")

In [7]:
def subwindow(array, i:int, j:int, l:int):
    if i < 0 or j < 0:
        raise "Coordinates must be within the margins of the array"
    if (i + l) >= array.shape[0] or (j + l) >= array.shape[1]:
        return

    return array[i:(i + l), j:(j + l)]

In [8]:
def integral_image(image: np.array) -> tf.Tensor:
    """Compute the integral image of the given image.
    
    Parameters:
    -----------
    image: `tf.Tensor`
        The image to compute the integral image of.
    """

    int_img:np.array = np.cumsum(
        np.cumsum(image, axis = 1),
        axis = 0
    )
    return tf.cast(int_img, dtype = tf.float32)

(480, 640, 3)


In [11]:
def variance_normalize(int_img: tf.Tensor, img: tf.Tensor):
    x = tf.reshape(int_img, [-1])
    x2_sum = tf.reduce_sum(tf.pow(x, 2))
    n = int_img.shape[0]
    m = int_img.shape[1]
    N = n * m
    mean = float(int_img[n - 1, n - 1] / N)
    
    print(N, mean**2, x2_sum * (1/N))
    var = mean**2 - x2_sum * (1/N)
    print(var)
    normalized_img = tf.cast(int_img, dtype = tf.float32)
    normalized_img *= (1/(var ** 0.5))
    return normalized_img

In [10]:
img = load_img("example.jpg", SCAN_RESOLUTION)
img = to_grayscale(img)
int_img = integral_image(img)
sub_int_img = subwindow(int_img, 0, 0, 5)
sub_img = subwindow(img, 0, 0, 5)
# var_nor = variance_normalize(sub_int_img, sub_img)
# plt.imshow(img)
sub_img, sub_int_img

(array([[64, 66, 68, 63, 66],
        [66, 64, 61, 66, 72],
        [68, 64, 64, 69, 70],
        [74, 69, 67, 67, 66],
        [73, 70, 68, 66, 68]], dtype=uint32),
 <tf.Tensor: shape=(5, 5), dtype=float32, numpy=
 array([[  64.,  130.,  198.,  261.,  327.],
        [ 130.,  260.,  389.,  518.,  656.],
        [ 198.,  392.,  585.,  783.,  991.],
        [ 272.,  535.,  795., 1060., 1334.],
        [ 345.,  678., 1006., 1337., 1679.]], dtype=float32)>)

In [261]:
tf.reduce_sum(sub_img ** 2)

<tf.Tensor: shape=(), dtype=uint32, numpy=219094>

In [90]:
img = [
    [1,7,4,2,9],
    [7,2,3,8,2],
    [1,8,7,9,1],
    [3,2,3,1,5],
    [2,9,5,6,6],
]
integral_image(img)

<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
array([[  1,   8,  12,  14,  23],
       [  8,  17,  24,  34,  45],
       [  9,  26,  40,  59,  71],
       [ 12,  31,  48,  68,  85],
       [ 14,  42,  64,  90, 113]])>

In [10]:
def haar_wavelet(t):
    if 0 <= t < 0.5:
        return 1
    elif 0.5 <= t < 1:
        return -1
    return 0

In [9]:
def haar_filter():
    pass

In [16]:
def initialize_weights(n:int, m:int, l:int):
    """Initialize the classifier's weights
    
    Parameters:
    -----------
    n: `int`
        number of elements in the training set 
    m: `int`
        number of negative examples in the training set
    l: `int`
        number of positive examples in the training set
    """

    w1: float = 1 / (2 * m)
    w2: float = 1 / (2 * l)

    weights_array = np.zeros((n, 2))
    weights_array[:] = w1, w2

    weights = tf.Variable(
        initial_value= weights_array,
        shape = (n, 2),
        dtype = tf.float32
    )

    return weights

In [17]:
initialize_weights(10, 3, 1)

<tf.Variable 'Variable:0' shape=(10, 2) dtype=float32, numpy=
array([[0.16666667, 0.5       ],
       [0.16666667, 0.5       ],
       [0.16666667, 0.5       ],
       [0.16666667, 0.5       ],
       [0.16666667, 0.5       ],
       [0.16666667, 0.5       ],
       [0.16666667, 0.5       ],
       [0.16666667, 0.5       ],
       [0.16666667, 0.5       ],
       [0.16666667, 0.5       ]], dtype=float32)>

In [6]:
def weighted_MAE(w, y_true, y_pred):
    return tf.sum(w * tf.abs(y_pred - y_true))

In [ ]:
def train(features):
    j:int = len(features)
    